In [28]:
import chromadb
import asyncio
import ollama
import os
import uuid
import json

## This block is for "global-variable" setup

In [2]:
token_size = 512
shift_size = 400

## This block is for "ChromaDB" setup

In [3]:
collection = "docs"
chroma_client = chromadb.HttpClient(host='localhost', port=8000)
collection = chroma_client.get_or_create_collection(name=collection)

def heartbeat():
    return chroma_client.heartbeat()

def collection_add(i, embedding, document, metadata):
    collection.add(
        ids=[i],
        embeddings=[embedding],
        documents=[document],
        metadatas=[metadata]
    )

def collection_query(query, n_results):
    results = collection.query(
        query_embeddings=query,
        n_results=n_results
    )
    return results

def collection_reset():
    while True:
        data_ids = collection.peek()['ids']
        if not data_ids:
            break
        for i in data_ids:
            # print(i)
            collection.delete(i)

## This block is for "DB like controller"

In [5]:
def load_status(filename):
    # 檢查文件是否存在且不為空
    if not os.path.exists(filename) or os.stat(filename).st_size == 0:
        return {}
    with open(filename, 'r', encoding='utf-8') as f:
        try:
            return json.load(f)
        except json.JSONDecodeError:
            # 如果 json 文件內容格式錯誤，返回空字典
            return {}

def save_status(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

sql_file = 'sql.json'
sql_data = load_status(sql_file)

## This block is for "RAG pipeline related" funciton
- prompt_expansion: use LLM to gen more related prompt to evaluate the response precision

In [6]:
# LLM prompt expansion >> 類似使用 "many-hit" 的方式來提高搜尋命中率
def prompt_expansion(query):
    expansive_prompts = ollama.generate(
        model="llama3",
        prompt=f"請用繁體中文，以這個 prompt: '{query}' 為基礎，以 array list 的方式，產生出額外 5 個相關的提問 prompt，我需要的 response 格式為 ['第一個產生的相似提問', '第二個產生的相似提問', '第三個產生的相似提問'] 這樣即可，不需要額外的其他內容，注意，相似提問的內容請以繁體中文呈現為主"
    )
    return expansive_prompts['response']

def query_rerank(embedding, topk):
    results = collection_query(embedding, topk)
    return results

## This block is for Application layer

In [7]:
path2dataset = './dataset/'
def read_documents_from_directory(directory):
    documents = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                documents.append(file.read())
                filenames.append(filename)
    return documents, filenames

documents, filenames = read_documents_from_directory(path2dataset)

In [17]:
query = "歐洲遊學"
# expansive_prompts = prompt_expansion(query)
# print(f"extend: {expansive_prompts}")

In [18]:
embedding = ollama.embeddings(prompt=query,model="llama3")["embedding"]

In [19]:
topk = 3
results = collection_query(embedding, topk)
# print(results)
tmp_query_result = []
for d in results['documents']:
    tmp_query_result.append(d)

print(f"tmp_query_result: {tmp_query_result}")
query_justify = ollama.generate(
    model="llama3",
    prompt=f"It's a YES/No question. 如果下列於 array list [] 內的內容，有任何可以是這個問題: '{query}' 的答案，請回答 yes,否則請回答 not"
)
print(query_justify['response'])

tmp_query_result: [['意義不太大不知道發論文,所以我很早就知道幹我超討厭我超討厭盧培和所以我從來都不太會真的不太會,就很煩,都是數學然後,因為我覺得研究比較像是單點突破你要把一個細節把它做出一些新的東西可是從工程師的角度,比如說以軟體工程的角度你要去優化一個系統你有非常多選擇那這個就變成說你今天是一個,Solver Engineer是一個total solution,但你做research變成說你在某一個很特別的一個part,你必須要挖得比別人更深所以我沒辦法做應該說,其實我也不太喜歡那請問子緣,你當時是發現自己有這方面的一個天賦或興趣嗎?其實講到這個我就有點有點不好意思因為說真的,我覺得說天賦嗎?我其實沒有很有天賦,我有看過那種真的研究狂人,他是真的就是無時無刻不在做研究然後我覺得我沒有那麼我不排斥,但是我也沒有到真的很狂熱那我覺得真的會想要念博士主要還是就是有我一些想做的事情然後再加上自己其實我會喜歡看著這個東西,然後想要去把它搞懂那這個搞懂的過程,其實就像Ted說的一樣可能像工作你有很多他是一個total solution,那其實研究某方面來說同樣的問題也有很多種不同的解決方式某種程度來說這也是一種total solutio', '但是他會先讓你有一個picture讓你知道說要到那個階段大概要經歷過哪一些事情,然後你可能要做哪些準備真的是去走一條就是有些人已經幫你除過草你可以在那裡可以稍走一點彎路這也是我們創這些做這些內容分享很重要的一個原因大部分的人除非你今天是馬斯克,你要射火箭沒有人射過,不然大部分樓部都有人走過所以如果你有人可以參考那當然是很好的一件事情我們會把子緣的聯絡方式都放在下面大家有興趣可以關注他的內容那我自己覺得大家可以可能每個人都有每個人不同的一個生活的軌跡但你或許沒辦法帶走他每個部分但你可以帶走他的一個態度我覺得子緣他在很多做事的方式大家可以參考因為我們認識很久,我知道他是一個非常努力的人那這種態度是我自己認為你或許沒有人家聰明你或許沒有人家有資源但是你可以比別人做得更努力那你一樣有機會拿到你喜歡的一個成果那我知道就是在這些大家在找這些資料的時候其實你很難去找到好的管道那我剛剛說了如果有錢人走過,你有一些路可以參考那我認為是蠻重要的那最後可以請子緣幫我們介紹你的自媒體你的一些分享內容大家可以怎麼樣找到你我現在自己就是對,

In [38]:
test_content = "大家好,歡迎來到成人頻道,最真實的工程師Podcast在這裡會討論最新的科技發展、產業趨勢以及工程師質素分享喜歡本頻道的聽眾請到Echo Podcasts還有Spotify上面給我們五星的評價留言也分享給更多的朋友,讓我們把想法傳遞給更多的人嗨大家好,歡迎來到今天的節目,我是Ted今天這個開頭,我會講一下我自己的想法我自己的想法,我自己的想法我自己的想法,我自己的想法我自己的想法,我自己的想法大家好,我是Ted今天這個開頭,我已經吃了兩次螺絲其實剛剛發現好像也沒有錄好,但沒插算的反正我們已經一陣子沒有錄音了但因為頻道最近在做一些我們希望在內容上的一些調整以及我們重新去找一些來賓等等還有我們之後的一些計畫很快就會跟大家公佈但我們還是一樣,只要有好的內容還有一些很酷的來賓他們的經歷我們都希望能夠邀請他們與大家一起分享今天這位來賓是我自己算是有一次因緣際會看到他的表演我覺得非常非常有趣那也很有這個榮幸可以來邀請到他與我們聊聊他的經歷那我們事不宜遲,先歡迎今天的來賓,藍恩嗨,大家好,我是藍恩我覺得這個是我等一下再跟大家講為什麼我會這麼的開心有這個機會但在開始之前還是請藍恩先幫我們向觀眾介紹一下自己藍恩這個名字比較主要是用在喜劇院上然後反正我平常的工作是現在在微軟當軟體工程師然後我帶的Team是Power BI Team就是產品是Power BI然後Power BI底下的一個做Micro Service的Team叫做Analysis Service Team酷酷酷那剛剛聽到一個關鍵字喜劇演員我就是在那個有一次在那個多多人的那個宴他就是在蘋果那一位嘛然後我就有看到他們的一個笑話上的互動然後聽到這個笑話就覺得這個絕對是我們說什麼這是絕對是同行才能夠寫出來的嘛一定不是外面的一個路邊的人可以寫出來然後我就會找說那到底是哪一位表演者這麼有才那我們就發現了藍恩的表演然後我也看了一些他在YouTube上面的影片我覺得相當的酷可以跟大家分享一下說當時就是你這樣穿插這是一下子工程師一下子喜劇演員你覺得你的生活就是現在的比重是什麼哇現在的比重我覺得就經歷上來說的話可能是maybe是2比1嘛就是工作2然後喜劇1但是反正我心態上好像比較偏一個槓鈴策略就是反正我90%會壓在工作上就是確保工作是就是在一個合理的順利的穩定的進行的狀態下然後10%就是壓在喜劇這邊就是他很花精力但是獲得的金錢上的回報很少但是蠻有趣的所以就一直想要去做他這樣你的那個認識我的路徑真的是非常特別就是我真的是史上唯一一個就是透過Dudu Man的那個Rose1n的笑話發現這件事情蠻特別的沒有想過還有這個路線可是我們一聽到就知道說你知道有些人家不是說什麼是什麼行啊來之類的反正就是我們一聽到就覺得這個一定是你一定是要懂一點東西你才有辦法你才有辦法講出這些東西嘛然後就天線都翹起來我們就說可以了解一下其實確實我們剛剛在開始錄音之前跟藍暗聊了一下就覺得真的就是很其實我就覺得很發現就是很很像同溫層相近的可能我們在學生時候還是出社會可能遇到的人也都是很類似的人因為包括你剛剛提到槓鈴策略這也是我們自己一直以來都在強調我們在投資上也都是用槓鈴策略在做就是一定要讀一些報集嘛然後這種媒體某一個程度就是槓鈴的比端嘛就突然可能一些莫名其妙好處從天上掉下來之類的這個就是90%壓大盤一帖然後10%跑去買那個薰衣貨幣大概就是類似這種感覺對對對但我的槓鈴比較大便宜一點我的薰衣貨幣不只10%當然大家講的沒錯對這其實就是這個概念那我覺得其實在生活中槓鈴的一端就保保你自己不進公園但是另外一端其實就是生活中不論是你金錢上的報集還是你體驗上的報集還是各種你意想不到的好事可能會是出現在那一個端點那我們還是今天這一集會著重在工程師相關那當然我們本身是創作者也很希望可以跟藍恩聊聊更多那會在下一集再跟大家分享那可以請藍恩跟大家介紹一下你在微軟的角色跟你日常的工作職責大概是哪些嗎好我講的比剛剛的詳細一點就是反正現在微軟基本上就是軟體工程師就是做的事情就是一般的規劃feature然後寫code這樣然後剛說做的是analysis service就是整個power BI這個做這個軟體以防大家不知道什麼是power BI總之你可以把它想像成如果從來沒聽過的話你可以想像成是Excel不是有一個就是畫圖表的功能嗎然後你把那個功能複雜大概100倍就是power BI的長相就是它做的事情就是主要是focus在資料視覺化的部分然後它複雜的點是在於說你可以同時導入很多個data source就不同的data base或者是你想要從網頁裡面去撈資料或者是你有一些檔案類的或者是你local的file它可以透過gateway再forward到power BI上就是反正資料來源很複雜然後你可以對圖表做的權限控管也很複雜所以總之就是一切都可以高度客製化的狀態所以就是反正它是一個500多個人500多個RD在開發的一個軟體這樣然後我負責底下的這個micro service叫analysis service它主要是負責跟data source去做接觸就是我們會把data source的就是不同data source的東西拉進來然後在memory裡面形成一個方便query的cache就是以效能為核心的一個memory的solution這樣然後我們也有一個analysis service自己有一個query的語言叫做dex然後反正我們要去支援這樣的查詢這樣然後analysis service這個服務它已經超級久了大概20年就是它跟第一代的SQL service或是Excel是一起出現所以這個codebase已經超過20年但它被用在各式各樣不同的地方這樣所以反正基本上就是在裡面去開發新的feature這樣然後另外一件事情好像可以提的是我們每兩個月大概會要值班一週那那個值班基本上就是你會當第三線的客服就是你要負責去解就是客戶被Excel上來的ticket就是他遇到一些很奇怪的issue然後一般的客服解不了你就要負責看這樣然後我們也有很多什麼monitoring alert這種你也要負責去排除就是去看是false alarm還是真流regression這樣了解剛剛提到幾個點我這有幾個問題想問我先跟大家講如果大家還不了解power BI的話你可以去參考陳真頻道的211幾那Stark有來分享他前陣子剛有出了一本書那剛剛提到說你現在處理這個服務的codebase已經超過已經有20年那會有很多技術債嗎就是比如說你們要開發新的因為剛剛提到現在是micro service然後micro service這個概念顯然是比較後期的東西那在前期那種打爆一整炮的是不是有點不太好讓你們拿出來弄就是技術債的話我自己體感是覺得會有一些東西被拖著但不會有就是沒有人就是沒有人知道那個地方要怎麼動就是這個倒還好的原因是因為他雖然超過20年但我們聽也有超就是好幾個是你知道從最一開始就在20幾年然後他們就那開始的codebase甚至是C就他還不是C++他是C所以就是現在還是可以看到我們code裡面有很多用一大堆macro然後那些macro拼拼湊湊其實就是C++的某一個modernize的功能所以其實我們除了在推新的feature之外一直有一個就是在background在做的事情是去modernize很多東西同時在reflect一些舊的東西這樣對對對但我比較少參與modernize的那個部分因為就是他比較需要對C的那一塊codebase很了解的人來進行但總之比較新的大概都是用C++C sharp在弄了解如果這個轉換聽起來就很痛苦可能你要從C的概念然後再轉到OOP的這種東西你是要對兩個語言都需要很了解真的而且C在那邊指來指去指到不知道指到哪邊了我自己在看的時候我就是覺得很你知道在看一個我不知道IG金字塔的感覺就是他用一堆macro去堆出比如說他自動free東西就是OOP裡面會有的概念然後他用C去把它硬刻出來其實也蠻屌的對這個東西我要帶出來稍微分享一下就是我永遠記得有一次我跟那個就是我以前學校學長姐去錄應該兩三年前然後就是有那個比較偏機械演藝的學長他們就在那邊分享他現在在寫程式然後他就有一個人就說其實C也可以寫出C++的那些什麼繼承另外一個就跳出來說一定可以啊然後在那邊分享我心裡想說靠北啊我當然知道可以啊有沒有人會這樣用而已啊真的就是如果你現在2024年還在做什麼事情這真的自找麻煩耶我覺得對啊就是有時候真的是一兩句話要聽到這個人是不是來搞笑的不過這也是感覺還是蠻酷的一個經歷"
content_splitter = ollama.generate(
    model="llama3",
    prompt=f"需求：請以這段 content:'{test_content}' 為基礎，以 array list 的方式，對這個 content 做 data chunk。Response： ['第一段chunk','第二段chunk','第三段chunk',.....'第Ｎ段chunk']"
)

print(content_splitter['response'])

Here is the segmented content in an array list format:

['大家好,歡迎來到成人頻道, source的東西拉進來然後在memory裡面形成一個方便query的cache就是以效能為核心的一個memory的solution這樣然後我們也有一個analysis service自己有一個query的語言叫做dex然後反正我們要去支援這樣的查詢這樣然後analysis service這個服務它已經超級久了大概20年就是它跟第一代的SQL service或是Excel是一起出現所以這個codebase已經超過20年但它被用在各式各樣不同的地方']

['另外一件事情好像可以提的是我們每兩個月大概會要值班一週那那個值班基本上就是你會當第三線的客服就是你要負責去解就是客戶被Excel上來的ticket就是他遇到一些很奇怪的issue然後一般的客服解不了你就要負責看這樣']

['我們也有很多什麼monitoring alert這種你也要負責去排除就是去看是false alarm還是真流regression這樣了解剛剛提到幾個點我這有幾個問題想問']

['如果大家還不了解power BI的話你可以去參考陳真頻道的211幾那Stark有來分享他前陣子剛有出了一本書']

['現在處理這個服務的codebase已經超過已經有20年那會有很多技術債嗎就是比如說你們要開發新的因為剛剛提到現在是micro service然後micro service這個概念顯然是比較後期的東西']

['在前期那種打爆一整炮的是不是有點不太好讓你們拿出來弄就是技術債的話我自己體感是覺得會有一些東西被拖著但不會有就是沒有人就是沒有人知道那個地方要怎麼動']

['就是這個倒還好的原因是因為他雖然超過20年但我們聽也有超就是好幾個是你知道從最一開始就在20幾年然後他們就那開始的codebase甚至是C就他還不是C++他是C所以就是現在還是可以看到我們code裡面有很多用一大堆macro然後那些macro拼拼湊湊其實就是C++的某一個modernize的功能']

['這樣對對對但我比較少參與modernize的那個部分因為就是他比較需要對C的那一塊codebase很了解的人來進行']

['總之比較新的大概都是用C++C sharp在弄了解如果這個轉換聽

In [ ]:
def process_document(filename, doc, model="llama3"):
    print(f"file: {filename}")
    length = len(doc)
    start = 0
    embeddings = []
    chunks = []
    i = 0

    while start < length:
        end = min(start + token_size, length)
        chunk = doc[start:end]
        # print(f"chunk: {chunk}")
        sumerize = ollama.chat(
            model = model,
            messages= [
                {
                    "role": "user",
                    "content": "我要你的回答只能是 Array-List object"
                },
                {
                    "role": "assistant",
                    "content": "[]"
                },
                {
                    "role": "user",
                    "content": f"很好，請針對下列 content:'{chunk}，產生一個 Array-List 包含屬於 content 的七個 關鍵字"
                }
            ]
        )
        print(f"sumerize: {sumerize['message']}")
        chunk_to_embedding = sumerize['message']['content']
        chunk_metadata = {"type":"podcast","name":"techporn","title":filename,"documents":chunk,"ids":i}
        # print(f"{chunk_to_embedding}")
        print(f"metadata: {chunk_metadata}")
        uid = uuid.uuid4()
        # print(f"ids:{uid}")
        print("--"*20)
        
        response = ollama.embeddings(model=model, prompt=chunk_to_embedding)
        embedding = response["embedding"]
        chunks.append(chunk_to_embedding)
        collection_add(str(uid), embedding, chunk, chunk_metadata)
        start += shift_size  # Move start forward by the shift size
        i += 1
    
    return chunks


# for i, (f, d) in enumerate(zip(filenames, documents)):
#     print(f)
#     process_document(f, d)
for i, (f, d) in enumerate(zip(filenames, documents)):
    if sql_data.get(f, {}).get('status') != 'done':
        process_document(f, d)
        file_path = os.path.join(path2dataset, f)
        sql_data[f] = {'filename': f, 'path': file_path, 'status': 'done'}
    else:
        print(f"...Skipping [DONE] - {f}")

save_status(sql_file, sql_data)